# Adversarial Search: Solving Tic-Tac-Toe with Monte Carlo Tree Search

## Introduction 

Multiplayer games can be implemented as:
1. Nondeterministic actions: The opponent is seen as part of an environment with nondeterministic actions. Non-determinism is the result of the unknown opponent's moves. 
2. Optimal Decisions: Minimax search (search complete game tree) and alpha-beta pruning.
3. Heuristic Alpha-Beta Tree Search: Cut off tree search and use heuristic to estimate state value. 
4. __Monte Carlo Search:__ Simulate playouts to estimate state value. 

Here we will implement search for Tic-Tac-Toe (see [rules](https://en.wikipedia.org/wiki/Tic-tac-toe)). The game is a __zero-sum game__: Win by x results in +1, win by o in -1 and a tie has a value of 0. Max plays x and tries to maximize the outcome while Min plays o and tries to minimize the outcome.   

We will implement
* Pure Monte Carlo search

## The board

I represent the board as a vector of length 9. The values are `' ', 'x', 'o'`.  

In [1]:
import numpy as np
import math

In [2]:
def empty_board():
    return [' '] * 9

board = empty_board()
display(board)

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']

Some helper functions.

In [3]:
def show_board(board):
    """display the board"""
    b = np.array(board).reshape((3,3))
    print(b)

board = empty_board()
show_board(board)    

print()
print("Add some x's")
board[0] = 'x'; board[3] = 'x'; board[6] = 'x';  
show_board(board)

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]

Add some x's
[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]


In [4]:
def check_win(board):
    """check the board and return one of x, o, d (draw), or n (for next move)"""
    
    board = np.array(board).reshape((3,3))
    
    diagonals = np.array([[board[i][i] for i in range(len(board))], 
                          [board[i][len(board)-i-1] for i in range(len(board))]])
    
    for a_board in [board, np.transpose(board), diagonals]:
        for row in a_board:
            if len(set(row)) == 1 and row[0] != ' ':
                return row[0]
    
    # check for draw
    if(np.sum(board == ' ') < 1):
        return 'd'
    
    return 'n'

show_board(board)
print('Win? ' + check_win(board))

print()
show_board(empty_board())
print('Win? ' + check_win(empty_board()))

[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]
Win? x

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]
Win? n


In [5]:
def get_actions(board):
    """return possible actions as a vector ot indices"""
    return np.where(np.array(board) == ' ')[0].tolist()

    # randomize the action order
    #actions = np.where(np.array(board) == ' ')[0]
    #np.random.shuffle(actions)
    #return actions.tolist()


show_board(board)
get_actions(board)

[['x' ' ' ' ']
 ['x' ' ' ' ']
 ['x' ' ' ' ']]


[1, 2, 4, 5, 7, 8]

In [6]:
def result(state, player, action):
    """Add move to the board."""
    
    state = state.copy()
    state[action] = player
  
    return state

show_board(empty_board())

print()
print("State for placing an x at position 4:")
show_board(result(empty_board(), 'x', 4))

[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]

State for placing an x at position 4:
[[' ' ' ' ' ']
 [' ' 'x' ' ']
 [' ' ' ' ' ']]


In [7]:
def other(player): 
    if player == 'x': return 'o'
    else: return 'x'

In [8]:
def utility(state, player = 'x'):
    """check is a state is terminal and return the utility if it is. None means not a terminal mode."""
    goal = check_win(state)        
    if goal == player: return +1 
    if goal == 'd': return 0  
    if goal == other(player): return -1  # loss is failure
    return None # continue

print(utility(['x'] * 9))
print(utility(['o'] * 9))
print(utility(empty_board()))

1
-1
None


# Pure Monte Carlo Search

See AIMA page 161ff. 

We implement a extremely simplified version.

For the current state: 
1. Simulate $N$ random playouts for each possible action and 
2. pick the action with the highest average utility.

__Important note:__ we use here a random playout policy, which ends up creating just a randomized search that works fine for this toy problem. For real applications you need to extend the code with
1. a good __playout policy__ (e.g., learned by self-play) and 
2. a __selection policy__ (e.g., UCB1).

## Simulate playouts

In [9]:
def playout(state, action, player = 'x'):
    """Perfrom a random playout starting with the given action on the fiven board 
    and return the utility of the finished game."""
    state = result(state, player, action)
    current_player = other(player)
    
    while(True):
        # reached terminal state?
        u = utility(state, player)
        if u is not None: return(u)
        
        # we use a random playout policy
        a = np.random.choice(get_actions(state))
        state = result(state, current_player, a)
        #print(state)
        
        # switch between players
        current_player = other(current_player)


board = empty_board()
print(playout(board, 0))
print(playout(board, 0))
print(playout(board, 0))

1
1
-1


In [10]:
def playouts(board, action, player = 'x', N = 100):
    """Perform N playouts following the given action for the given board."""
    return [ playout(board, action, player) for i in range(N) ]

p = playouts(board, 0)
print(p)

print(f"mean utility: {np.mean(p)}")
print(f"win probability: {sum(np.array(p) == +1)/len(p)}")
print(f"loss probability: {sum(np.array(p) == -1)/len(p)}")

[1, 1, 0, 1, -1, 1, 1, 0, 1, 1, -1, -1, -1, 0, -1, -1, -1, 1, 1, 0, -1, -1, 1, 1, -1, 1, 1, 1, 1, -1, -1, 1, 1, -1, 1, 1, 0, 0, 1, 1, -1, 0, 0, 1, 0, -1, 1, 1, -1, 1, -1, 0, 1, 1, 1, -1, 1, 1, -1, 1, 1, 0, 1, 0, -1, 1, -1, 1, -1, 0, 1, 1, 0, 1, 1, -1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, -1, -1, -1, 1, 1, 0, 1, 1, 1, 0, 1, -1, 1]
mean utility: 0.27
win probability: 0.54
loss probability: 0.27


__Note:__ This shows that the player who goes first has a significant advantage in __pure random play.__ A better playout policy would be good.

## Pure Monte Carlo Search

In [11]:
DEBUG = 1

def pmcs(board, N = 100, player = 'x'):
    """Pure Monte Carlo Search. Returns the action that has the largest average utility.
    The N playouts are evenly divided between the possible actions."""
    global DEBUG
    
    actions = get_actions(board)
    n = math.floor(N/len(actions))
    if DEBUG >= 1: print(f"Actions: {actions} ({n} playouts per actions)")
    
    ps = { i:np.mean(playouts(board, i, player, N = n)) for i in actions }
    if DEBUG >= 1: display(ps)
        
    action = max(ps, key=ps.get)
    return action

board = empty_board()
display(board)
%time print(pmcs(board))

print()
print("10000 playouts give a better utility estimate.")
%time print(pmcs(board, N = 10000))

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']

Actions: [0, 1, 2, 3, 4, 5, 6, 7, 8] (11 playouts per actions)


{0: 0.36363636363636365,
 1: 0.0,
 2: 0.6363636363636364,
 3: 0.0,
 4: 0.36363636363636365,
 5: 0.2727272727272727,
 6: 0.0,
 7: -0.18181818181818182,
 8: 0.5454545454545454}

2
CPU times: user 85.1 ms, sys: 1.12 ms, total: 86.2 ms
Wall time: 84.5 ms

10000 playouts give a better utility estimate.
Actions: [0, 1, 2, 3, 4, 5, 6, 7, 8] (1111 playouts per actions)


{0: 0.35733573357335735,
 1: 0.18181818181818182,
 2: 0.342034203420342,
 3: 0.20252025202520252,
 4: 0.5148514851485149,
 5: 0.15751575157515751,
 6: 0.38883888388838883,
 7: 0.16471647164716471,
 8: 0.3321332133213321}

4
CPU times: user 6.09 s, sys: 5.08 ms, total: 6.1 s
Wall time: 6.1 s


Looks like the center and the corners are a lot better.

## Some Tests

In [12]:
# x is about to win (play 8)

board = empty_board() 
board[0] = 'x'
board[1] = 'o'
board[3] = 'o'
board[4] = 'x'

print("Board:")
show_board(board)

print()
%time display(pmcs(board))

Board:
[['x' 'o' ' ']
 ['o' 'x' ' ']
 [' ' ' ' ' ']]

Actions: [2, 5, 6, 7, 8] (20 playouts per actions)


{2: 0.9, 5: 0.8, 6: 0.85, 7: 0.7, 8: 1.0}

8

CPU times: user 38.6 ms, sys: 233 µs, total: 38.8 ms
Wall time: 37.7 ms


In [13]:
# o is about to win

board = empty_board() 
board[0] = 'o'
board[1] = 'o'
board[3] = 'o'
board[4] = 'x'
board[8] = 'x'

print("Board:")
show_board(board)

print()
%time display(pmcs(board))

print()
%time display(pmcs(board, N = 1000))

Board:
[['o' 'o' ' ']
 ['o' 'x' ' ']
 [' ' ' ' 'x']]

Actions: [2, 5, 6, 7] (25 playouts per actions)


{2: 0.12, 5: -0.76, 6: -0.28, 7: -0.76}

2

CPU times: user 38.1 ms, sys: 3.35 ms, total: 41.4 ms
Wall time: 39.2 ms

Actions: [2, 5, 6, 7] (250 playouts per actions)


{2: -0.048, 5: -0.64, 6: 0.008, 7: -0.712}

6

CPU times: user 229 ms, sys: 96 µs, total: 229 ms
Wall time: 227 ms


In [14]:
#### x can draw if it chooses 7.

board = empty_board() 
board[0] = 'x'
board[1] = 'o'
board[2] = 'x'
board[4] = 'o'

print("Board:")
show_board(board)

print()
%time display(pmcs(board))

Board:
[['x' 'o' 'x']
 [' ' 'o' ' ']
 [' ' ' ' ' ']]

Actions: [3, 5, 6, 7, 8] (20 playouts per actions)


{3: 0.25, 5: 0.45, 6: -0.55, 7: 0.3, 8: 0.05}

5

CPU times: user 45.5 ms, sys: 132 µs, total: 45.6 ms
Wall time: 43.7 ms


In [15]:
# o went first

board = empty_board() 
board[4] = 'o'

print("Board:")
show_board(board)

print()
%time display(pmcs(board))

Board:
[[' ' ' ' ' ']
 [' ' 'o' ' ']
 [' ' ' ' ' ']]

Actions: [0, 1, 2, 3, 5, 6, 7, 8] (12 playouts per actions)


{0: -0.5833333333333334,
 1: -0.4166666666666667,
 2: -0.5833333333333334,
 3: -0.25,
 5: -0.6666666666666666,
 6: -0.5,
 7: -0.5,
 8: -0.5}

3

CPU times: user 76.3 ms, sys: 39 µs, total: 76.3 ms
Wall time: 74 ms


In [16]:
# Empty board: Only a draw an be guaranteed

board = empty_board() 

print("Board:")
show_board(board)

print()
%time display(pmcs(board ))

Board:
[[' ' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]

Actions: [0, 1, 2, 3, 4, 5, 6, 7, 8] (11 playouts per actions)


{0: 0.6363636363636364,
 1: 0.18181818181818182,
 2: 0.18181818181818182,
 3: 0.6363636363636364,
 4: 0.45454545454545453,
 5: 0.18181818181818182,
 6: 0.2727272727272727,
 7: 0.0,
 8: 0.09090909090909091}

0

CPU times: user 87 ms, sys: 262 µs, total: 87.2 ms
Wall time: 84.7 ms


In [17]:
# A bad situation

board = empty_board() 
board[0] = 'o'
board[2] = 'x'
board[8] = 'o'

print("Board:")
show_board(board)

print()
%time display(pmcs(board))

Board:
[['o' ' ' 'x']
 [' ' ' ' ' ']
 [' ' ' ' 'o']]

Actions: [1, 3, 4, 5, 6, 7] (16 playouts per actions)


{1: -0.875, 3: -0.6875, 4: 0.1875, 5: -0.875, 6: -0.375, 7: -0.5625}

4

CPU times: user 39.2 ms, sys: 0 ns, total: 39.2 ms
Wall time: 37.8 ms


__Note:__ It looks like random player o is very unlikely to block x and take advantage of the trap by playing the bottom left corner!

## Experiments


### Baseline: Randomized Player

A completely randomized player agent should be a weak baseline.

In [18]:
def random_player(board, player = None):
    """Simple player that chooses a random empy square. player is unused"""
    return np.random.choice(get_actions(board))

show_board(board)
print()
%time random_player(board)

[['o' ' ' 'x']
 [' ' ' ' ' ']
 [' ' ' ' 'o']]

CPU times: user 52 µs, sys: 5 µs, total: 57 µs
Wall time: 59.6 µs


7

### The Environment

Implement the environment that calls the agent. The percept is the board and the action is move.

In [19]:
DEBUG = 1

def switch_player(player, x, o):
    if player == 'x':
        return 'o', o
    else:
        return 'x', x

def play(x, o, N = 100):
    results = {'x': 0, 'o': 0, 'd': 0}
    for i in range(N):
        board = empty_board()
        player, fun = 'x', x
        
        while True:
            a = fun(board, player)
            board = result(board, player, a)
            if DEBUG >= 1: print(f"Player {player} uses action {a}")
            
            win = check_win(board)
            if win != 'n':
                if DEBUG >= 1: print(f"Result {board} winner: {win}")
                results[win] += 1
                break
            
            player, fun = switch_player(player, x, o)   
 
    return results

### Random vs. Random

In [20]:
DEBUG = 0

%time display(play(random_player, random_player))

{'x': 58, 'o': 33, 'd': 9}

CPU times: user 74.8 ms, sys: 0 ns, total: 74.8 ms
Wall time: 73.8 ms


### Pure Monte Carlo Search vs. Random

In [21]:
def pmcs10_player(board, player = 'x'):
    action = pmcs(board, N = 10, player = player)
    return action

def pmcs100_player(board, player = 'x'):
    action = pmcs(board, N = 100, player = player)
    return action

def pmcs1000_player(board, player = 'x'):
    action = pmcs(board, N = 1000, player = player)
    return action


DEBUG = 1
print("PMCS vs. random:")
display(play(pmcs10_player, random_player, N = 1))

PMCS vs. random:
Actions: [0, 1, 2, 3, 4, 5, 6, 7, 8] (1 playouts per actions)


{0: 1.0, 1: -1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: -1.0, 7: 1.0, 8: -1.0}

Player x uses action 0
Player o uses action 7
Actions: [1, 2, 3, 4, 5, 6, 8] (1 playouts per actions)


{1: 1.0, 2: -1.0, 3: 1.0, 4: 1.0, 5: 1.0, 6: 0.0, 8: 1.0}

Player x uses action 1
Player o uses action 4
Actions: [2, 3, 5, 6, 8] (2 playouts per actions)


{2: 1.0, 3: 0.0, 5: -0.5, 6: 1.0, 8: 1.0}

Player x uses action 2
Result ['x', 'x', 'x', ' ', 'o', ' ', ' ', 'o', ' '] winner: x


{'x': 1, 'o': 0, 'd': 0}

In [22]:
DEBUG = 0
print("PMCS vs. random:")
%time display(play(pmcs10_player, random_player))

print()
print("random vs. PMCS")
%time display(play(random_player, pmcs10_player))

PMCS vs. random:


{'x': 88, 'o': 8, 'd': 4}

CPU times: user 1.29 s, sys: 3.83 ms, total: 1.29 s
Wall time: 1.29 s

random vs. PMCS


{'x': 29, 'o': 57, 'd': 14}

CPU times: user 921 ms, sys: 185 µs, total: 921 ms
Wall time: 921 ms


In [23]:
DEBUG = 0
print("PMCS vs. random:")
%time display(play(pmcs100_player, random_player))

print()
print("random vs. PMCS")
%time display(play(random_player, pmcs100_player))

PMCS vs. random:


{'x': 99, 'o': 0, 'd': 1}

CPU times: user 13.1 s, sys: 349 µs, total: 13.1 s
Wall time: 13.1 s

random vs. PMCS


{'x': 11, 'o': 79, 'd': 10}

CPU times: user 10.6 s, sys: 144 µs, total: 10.6 s
Wall time: 10.6 s


In [24]:
DEBUG = 0
print("PMCS (100) vs. PMCS (10):")
%time display(play(pmcs100_player, pmcs10_player))

print()
print("PMCS (10) vs. PMCS (100)")
%time display(play(pmcs10_player, pmcs100_player))

PMCS (100) vs. PMCS (10):


{'x': 87, 'o': 6, 'd': 7}

CPU times: user 14.1 s, sys: 88 µs, total: 14.1 s
Wall time: 14.1 s

PMCS (10) vs. PMCS (100)


{'x': 32, 'o': 56, 'd': 12}

CPU times: user 11.3 s, sys: 19 µs, total: 11.3 s
Wall time: 11.3 s


_Note 1:_ You can try `pmcs_100`, but it will take a few minutes to run 100 games.

_Note 2:_ The important advantage about Monte Carlo Search is that we do not need to define a heuristic evaluation function to decide what boards are good.